This notebook computes the power spectra of the modes of deformation of the critical curves.

Author: Julien Larena

# 1- Initialisation

In [ ]:
# Load packages, including CAMB
%matplotlib inline
import sys, platform, os
import matplotlib
from matplotlib import pyplot as plt
import matplotlib.pyplot as plt
# Use TeX
from matplotlib import rc
# rc('font',**{'family':'serif','serif':['Times']})
rc('text', usetex=True)
rc('font', family='serif')
#rc('font', **{'serif':['Times']})
matplotlib.rcParams.update({'font.size': 15})

from scipy import constants, special
import numpy as np
from scipy.interpolate import InterpolatedUnivariateSpline as Spline
from hankel import HankelTransform

print('Using CAMB installed at %s'%(os.path.realpath(os.path.join(os.getcwd(),'..'))))
#uncomment this if you are running remotely and want to keep in synch with repo changes
#if platform.system()!='Windows':
#    !cd $HOME/git/camb; git pull github master; git log -1
sys.path.insert(0,os.path.realpath(os.path.join(os.getcwd(),'..')))
import camb
from camb import model, initialpower
print('Using CAMB %s installed at %s'%(camb.__version__,os.path.dirname(camb.__file__)))

# Changing radians to arcsec
rad2arcsec=648*1000/np.pi


In [ ]:
# CAMB parameters
pars = camb.CAMBparams()
pars.set_cosmology(H0=67.5, ombh2=0.022, omch2=0.122)
pars.InitPower.set_params(ns=0.965)

In [ ]:
# Generate spectra up to very small scales (this is the longest step)
zmax = 5
kmax = 1e3 #(inverse Mpc)
Weyl_power_spectra = camb.get_matter_power_interpolator(pars,nonlinear=True,zmax=zmax, kmax=kmax, zs=None,hubble_units=False, k_hunit=False, var1=model.Transfer_Weyl,var2=model.Transfer_Weyl)

# 2- Window functions and kernels

In [ ]:
def WC(z,kappa,zd,zs):
    "This function returns the W_C window function calculated at a distance chi(z)"

    # Conformal distances and redshifts
    results= camb.get_background(pars)
    chi = results.conformal_time(0) - results.conformal_time(z)
    chid = results.conformal_time(0) - results.conformal_time(zd)
    chis = results.conformal_time(0) - results.conformal_time(zs)
    
    if chi>chis:
        wc=0
    else:
        if chi>=chid:
            wc=((chis-chi)/chis-(chis-chi)*(chi-chid)/((chis-chid)*chi))/(4*(1-kappa))
        else:
            wc=((chis-chi)/chis-(chid-chi)/chid)/(4*(1-kappa))                                               
    return wc

def WD(z,zd,zs):
    "This function returns the W_D window function calculated at a distance chi(z)"

    # Conformal distances and redshifts
    results= camb.get_background(pars)
    chi = results.conformal_time(0) - results.conformal_time(z)
    chid = results.conformal_time(0) - results.conformal_time(zd)
    chis = results.conformal_time(0) - results.conformal_time(zs)
    
    if chi>chis:
        wd=0
    else:
        if chi>=chid:
            wd=0
        else:
            wd=(chid-chi)/(2*chid)
    return wd

def q(n,z,l,kappa,zd,zs,epsilon):
    "Kernel inside the integrals defining the effective modes"
    # Conformal distances and redshifts
    results= camb.get_background(pars)
    chi = results.conformal_time(0) - results.conformal_time(z)
    chid = results.conformal_time(0) - results.conformal_time(zd)
    chis = results.conformal_time(0) - results.conformal_time(zs)
    
    if chi<=chid:
        beta=epsilon
    else:
        beta=epsilon*chid*(chis-chi)/((chis-chid)*chi)
    if n==1:
        q=WC(z,kappa,zd,zs)*(4*special.jn(n-1,l*beta)/(l*beta)-4*n*(n+1)*special.jn(n,l*beta)/(l*beta)**2)+WD(z,zd,zs)*(4*special.jn(n-1,l*beta)/(l*beta)-4*(n**2)*special.jn(n,l*beta)/(l*beta)**2-2/(l*beta))
    else:
        q=WC(z,kappa,zd,zs)*(4*special.jn(n-1,l*beta)/(l*beta)-4*n*(n+1)*special.jn(n,l*beta)/(l*beta)**2)+WD(z,zd,zs)*(4*special.jn(n-1,l*beta)/(l*beta)-4*(n**2)*special.jn(n,l*beta)/(l*beta)**2)
    return q


# 3- Create the spectra

In [ ]:
lsize=150   # Size of the mode sample
zsize=150   # Size of the redshift sample for integrals

ll=np.exp(np.log(10)*np.linspace(0,4,lsize))  # Modes where the spectra are calculated

zz=np.linspace(0,5,zsize)  # Redshift to perform integrals

## Realisation for parameters: Einstein rings, redshifts of sources.
## Note that lens redshifts are generated using in the loops below for each redshift of source generated to ensure physical system.

sample=10

# PDF of Einstein rings: Lognormal distribution
sEr=0.5 
mEr=np.log(0.8)-sEr**2+np.log(np.pi*0.001/648)   #using radians
thetareal=np.random.lognormal(mean=mEr,sigma=sEr,size=sample)

# PDF of source redshifts: Normal distribution
muzs=2
sigzs=0.6
zsreal=np.random.normal(loc=muzs,scale=sigzs,size=sample)

# PDF of lens redshifts: Rayleigh distribution
szd=0.5

# Values of the convergence at the Einstein Rings for which spectra are generated
npreal=5
preal=np.linspace(1.5,2.5,npreal)
kappareal=3/2-preal/2

# Initialisation
zdreal=np.zeros((npreal,sample,sample,sample))
Qq2=np.zeros((len(ll),len(zz),npreal,sample,len(zsreal),len(thetareal)))
Qq3=np.zeros((len(ll),len(zz),npreal,sample,len(zsreal),len(thetareal)))
#Qq4=np.zeros((len(ll),len(zz),npreal,sample,len(zsreal),len(thetareal)))

# Calculation of integrands

for p, kappa in enumerate(kappareal):
    for m, z in enumerate(zz):
        for i, theta in enumerate(thetareal):
            for j, zs in enumerate(zsreal):
                for k in range(sample):
                    zd=np.random.rayleigh(scale=szd,size=None)
                    while zd>zs:
                        zd=np.random.rayleigh(scale=szd,size=None)
                    zdreal[p,i,j,k]=zd    
                    Qq2[:,m,p,k,j,i]=q(2,z,ll,kappa,zd,zs,theta)
                    Qq3[:,m,p,k,j,i]=q(3,z,ll,kappa,zd,zs,theta)
#                    Qq4[:,m,p,k,j,i]=q(4,z,ll,kappa,zd,zs,theta)


# Calculation of ensemble averages on random parameters

Q2av=np.sum(Qq2, axis=(2,3,4))/sample**3
Q3av=np.sum(Qq3, axis=(2,3,4))/sample**3
#Q4av=np.sum(Qq4, axis=(2,3,4))/sample**3

In [ ]:
# Create the spectra

P22=np.zeros((len(ll),len(kappareal)))
P23=np.zeros((len(ll),len(kappareal)))
#P33=np.zeros((len(ll),len(kappareal)))
#P24=np.zeros((len(ll),len(kappareal)))
#P34=np.zeros((len(ll),len(kappareal)))
#P44=np.zeros((len(ll),len(kappareal)))
results= camb.get_background(pars)

chi=results.conformal_time(0) - results.conformal_time(zz)
dchi = (chi[2:]-chi[:-2])/2
chi = chi[1:-1] 
zzz = zz[1:-1]

Power=np.zeros((len(zzz),len(ll)))
for i, z in enumerate(zzz):
    for j, Ll in enumerate(ll):
        Power[i,j]=Weyl_power_spectra.P(z, Ll/chi[i], grid=False)

for i, l in enumerate(ll):
    for j,kappa in enumerate(kappareal):
        P22[i,j]=np.dot(Q2av[i,1:-1,j]*Q2av[i,1:-1,j]*Power[:,i],dchi)
        P23[i,j]=np.dot(Q2av[i,1:-1,j]*Q3av[i,1:-1,j]*Power[:,i],dchi)
#        P33[i,j]=np.dot(Q3av[i,1:-1,j]*Q3av[i,1:-1,j]*Power[:,i],dchi)
#        P24[i,j]=np.dot(Q2av[i,1:-1,j]*Q4av[i,1:-1,j]*Power[:,i],dchi)
#        P34[i,j]=np.dot(Q3av[i,1:-1,j]*Q4av[i,1:-1,j]*Power[:,i],dchi)
#        P44[i,j]=np.dot(Q4av[i,1:-1,j]*Q4av[i,1:-1,j]*Power[:,i],dchi)      

In [ ]:
# Save the spectra for future use: each value of \kappa_E corresponds to a column
np.savetxt('./data_spectra/P22.txt', P22, fmt='%e')
np.savetxt('./data_spectra/P23.txt', P23, fmt='%e')